In [ ]:
import requests
import json

In [ ]:
response = requests.get("http://api.pioupiou.fr/v1/live/410").text
response_info = json.loads(response)

print(type(response_info))
print(response_info)

In [ ]:
def loop():

    import sqlite3
    import requests
    
    station = [410, 432, 113]

    foil = []

    for e in station :
    
        requete = f"http://api.pioupiou.fr/v1/live-with-meta/{e}"

        response = requests.get(requete)
        
        response_dict = response.json()
    
        foil.append(response.json())
    
    db = sqlite3.connect('winds.db')

    cursor = db.cursor()

    for data in foil:
    
        data = data['data']
    
        print(data)
    
        cursor.execute("""INSERT INTO measurement (id,
                                                   latitude,
                                                   longitude,
                                                   state,
                                                   date,
                                                   wind_heading,
                                                   wind_speed_avg,
                                                   wind_speed_max,
                                                   wind_speed_min)
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)""",
                         (data['id'],
                          data['location']['latitude'],
                          data['location']['longitude'],
                          data['status']['state'],
                          data['measurements']['date'],
                          data['measurements']['wind_heading'],
                          data['measurements']['wind_speed_avg'],
                          data['measurements']['wind_speed_max'],
                          data['measurements']['wind_speed_min'])
                       )
                                        
        db.commit()
        
        print("Insertion effectuée")

    cursor.close()
    db.close()

In [ ]:
import schedule
import time

schedule.every(4).minutes.do(loop)

while True:
    schedule.run_pending()
    time.sleep(10)

In [24]:
def create_readable_dbcopy():
    
    #create a .sql file as a backup
    import sqlite3
    import io

    conn = sqlite3.connect('winds.db')
    with io.open('Backup/winds_dump.db', 'w') as f:
        for measurement in conn.iterdump():
            f.write('%s\n' % measurement)
            
    print('Backup performed successfully.')
    print('Saved as winds_dump.db')
    
    conn.close()

In [25]:
create_readable_dbcopy()

Backup performed successfully.
Saved as winds_dump.db


In [22]:
def create_copy_db():

    #create a copy of the database as a backup
    import shutil
    import os
 
    try:
        path='C:/Users/joris/Documents/Simplon/Dev_data/Vents/Foil'
    
        shutil.copyfile('winds.db','C:/Users/joris/Documents/Simplon/Dev_data/Vents/Foil/Backup/winds.db')
 
        print("Success copying winds.db")
    
    except:
        print("error while copying winds.db")

In [23]:
create_copy_db()

Success copying winds.db
